In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np

In [6]:
from pandas import DataFrame
data=pd.read_csv('share-urban-and-rural-population.csv')
print(data.head())


        Entity Code  Year  Urban population (% of total population)  \
0  Afghanistan  AFG  1960                                     8.401   
1  Afghanistan  AFG  1961                                     8.684   
2  Afghanistan  AFG  1962                                     8.976   
3  Afghanistan  AFG  1963                                     9.276   
4  Afghanistan  AFG  1964                                     9.586   

   Rural population (% of total population)  
0                                    91.599  
1                                    91.316  
2                                    91.024  
3                                    90.724  
4                                    90.414  


In [7]:
keepcolumn=['Entity','Year','Urban population (% of total population)']
cleandata=data[keepcolumn]
print(cleandata)

            Entity  Year  Urban population (% of total population)
0      Afghanistan  1960                                     8.401
1      Afghanistan  1961                                     8.684
2      Afghanistan  1962                                     8.976
3      Afghanistan  1963                                     9.276
4      Afghanistan  1964                                     9.586
...            ...   ...                                       ...
14422     Zimbabwe  2018                                    32.209
14423     Zimbabwe  2019                                    32.210
14424     Zimbabwe  2020                                    32.242
14425     Zimbabwe  2021                                    32.303
14426     Zimbabwe  2022                                    32.395

[14427 rows x 3 columns]


In [8]:
df1 = cleandata.dropna()
print(df1)
df2 = df1.drop_duplicates()
print(df2)


            Entity  Year  Urban population (% of total population)
0      Afghanistan  1960                                     8.401
1      Afghanistan  1961                                     8.684
2      Afghanistan  1962                                     8.976
3      Afghanistan  1963                                     9.276
4      Afghanistan  1964                                     9.586
...            ...   ...                                       ...
14422     Zimbabwe  2018                                    32.209
14423     Zimbabwe  2019                                    32.210
14424     Zimbabwe  2020                                    32.242
14425     Zimbabwe  2021                                    32.303
14426     Zimbabwe  2022                                    32.395

[14427 rows x 3 columns]
            Entity  Year  Urban population (% of total population)
0      Afghanistan  1960                                     8.401
1      Afghanistan  1961            

In [9]:
from scipy import stats
numeric_cols = df2.select_dtypes(include=[np.number])  # Select only numeric columns

# Step 2: Calculate Z-scores for numeric columns
z_scores = np.abs(stats.zscore(numeric_cols))

# Step 3: Filter out rows where the z-score is greater than 3 in any numeric column
# We use `.all(axis=1)` to keep rows where all z-scores are less than 3
df_filtered = df2[(z_scores < 3).all(axis=1)]

print("\nFiltered DataFrame (Outliers removed based on numeric columns):")
print(df_filtered)


Filtered DataFrame (Outliers removed based on numeric columns):
            Entity  Year  Urban population (% of total population)
0      Afghanistan  1960                                     8.401
1      Afghanistan  1961                                     8.684
2      Afghanistan  1962                                     8.976
3      Afghanistan  1963                                     9.276
4      Afghanistan  1964                                     9.586
...            ...   ...                                       ...
14422     Zimbabwe  2018                                    32.209
14423     Zimbabwe  2019                                    32.210
14424     Zimbabwe  2020                                    32.242
14425     Zimbabwe  2021                                    32.303
14426     Zimbabwe  2022                                    32.395

[14427 rows x 3 columns]


In [12]:
# Filter the data for the years 2012 to 2022
df_10_years = df_filtered[(df_filtered['Year'] >= 2012) & (df_filtered['Year'] <= 2022)]

# Calculate the mean urban population growth for each country over the past 10 years
df_mean_growth = df_10_years.groupby('Entity')['Urban population (% of total population)'].mean().reset_index()

# Filter the data to only keep the year 2022
df_2022 = df_filtered[df_filtered['Year'] == 2022]

# Merge the mean growth data with the 2022 data
df_2022 = df_2022.merge(df_mean_growth, on='Entity', suffixes=('', '_mean_growth'))

# Rename the new column for clarity
df_2022.rename(columns={'Urban population (% of total population)_mean_growth': 'Mean Growth (2012-2022)'}, inplace=True)

# Display the updated dataframe with the new column for mean growth
df_2022.head()
keepcolumn=['Entity','Urban population (% of total population)','Mean Growth (2012-2022)']
final=df_2022[keepcolumn]
print(final.head())

           Entity  Urban population (% of total population)  \
0     Afghanistan                                    26.616   
1         Albania                                    63.799   
2         Algeria                                    74.772   
3  American Samoa                                    87.196   
4         Andorra                                    87.811   

   Mean Growth (2012-2022)  
0                25.308909  
1                59.255091  
2                71.968636  
3                87.225455  
4                88.180273  
